<center><h1> Resolución de la Matriz de Khon-Shan </h1></center>
<center><h2> Proyecto Final Julia IIMAS ago 2021 </h2></center>
<center><h3> Autor: José Antonio Ayala Barbosa </h3></center>

#  Matriz de Khon-Shan

- La ecuación de Khon-Sham se usa para determinar, de manera aproximada, el nivel de energía más bajo de un sistema atómico.
- El método para su obtención, consiste en proponer una función tentativa que depende de varios parámetros, entre ellos la posicion de los electrones con respecto a los nucleos, los cuales se varían hasta que se tenga una energía mínima. 
- La ecuación es de tipo cuadrática, por lo que puede transformarse en una matriz cuadrática para encontrar su solución.

## Algoritmo numérico para su resolución

- Un método numerico utilizado para resolver la matriz de Khon-Sham, es el método de Jacobi.
- Matriz Khon-Sham tiene forma cuadrática (matriz simétrica o Hermitiana):
<div>
<img src="https://github.com/antonyayal/ProyectoFinalCursoJuliaAgo21/blob/4dea0a52ce0125dfe7018d61d843d7f81edcbc9c/Matriz.png" width="300"/>
</div>

## Objetivo. 
Encontrar:
- Los eigenvalores, tambien llamados valores caracteristicos de la matriz, son los escalares alojados en la diagonal principal (en caso de la matriz de Khon-Sham, estos escalares son las energías de los electrones del sistema atómico).
- Los eigenvectores, vectores columna a los que les corresponde un eigenvalor, (representan los estados en los que están los electrones).
<div>
<img src="https://github.com/antonyayal/ProyectoFinalCursoJuliaAgo21/blob/4dea0a52ce0125dfe7018d61d843d7f81edcbc9c/eigenvalores%20y%20eigenvectores.png" width="300"/>
</div>

## Diagonalización de matriz:
-Eliminar elementos fuera de diagonal para encontrar energias de los electrones.
<div>
<img src="https://github.com/antonyayal/ProyectoFinalCursoJuliaAgo21/blob/4dea0a52ce0125dfe7018d61d843d7f81edcbc9c/Rotacion%20matrices.png" width="300"/>
</div>

### Obtencion de eigenvectores por sustitución
<div>
<img src="https://github.com/antonyayal/ProyectoFinalCursoJuliaAgo21/blob/4dea0a52ce0125dfe7018d61d843d7f81edcbc9c/Sustitucion.png" width="300"/>
</div>

## Referencia
[1] Ayala-Barbosa, José Antonio. (2018) "Diagonalización de la matriz de Khon Sham...". Proyecto Final. Especialidad Cómputo de Alto Rendimiento". IIMAS UNAM. http://132.248.9.195/ptd2018/noviembre/0783467/Index.html

# Algortimo Jacobi

In [465]:
#Busca el máximo elemento en el triangulo superior
function max_element(mat)
    n=size(mat,1)
    piv_r = 1 
    piv_c = 2 #primera coordenada fuera de diagonal principal
    for r in 1:n-1
        for c in r+1:n
            if abs(mat[r,c]) > abs(mat[piv_r,piv_c])
                piv_r = r
                piv_c = c
            end
        end
    end
    return piv_r, piv_c
end
#a,b = max_element(mat)

max_element (generic function with 1 method)

In [466]:
#mat[a,b]

In [467]:
#Calculo de tangente
function cal_tang(mat,piv_r,piv_c)
    
    numerador = 2 * mat[piv_r,piv_c]
    
    a1 = mat[piv_r,piv_r] - mat[piv_c,piv_c]
    a2 = a1^2
    a3 = 4 * mat[piv_r,piv_c] * mat[piv_r,piv_c]
    
    denominador = a2 + a3
    denominador = √(denominador)
    denominador = abs(a1) + denominador
    return numerador/denominador
end

cal_tang (generic function with 1 method)

In [468]:
#Calculo de coseno
function cal_cose(tang)
    cose = 1 + tang^2
    cose = √(cose)
    cose = 1 / cose
    return cose
end

cal_cose (generic function with 2 methods)

In [469]:
#Calculo de seno
function cal_sino(cose,tang)
    return cose*tang
end

cal_sino (generic function with 1 method)

In [470]:
#Crea matriz T

function matrizT(mat,piv_r,piv_c)
    n=size(mat,1)
    tang = cal_tang(mat,piv_r,piv_c)
    cose = cal_cose(tang)
    sino = cal_sino(cose,tang)
    
    T = Matrix(1.0I, n, n)
    
    T[piv_r,piv_r] = cose
    T[piv_c,piv_c] = cose
    T[piv_r,piv_c] = -sino
    T[piv_c,piv_r] = sino
    return T
end
matrizT(mat,1,2)

10×10 Matrix{Float64}:
 0.707107  -0.707107  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.707107   0.707107  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0        0.0       1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0        0.0       0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0        0.0       0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0
 0.0        0.0       0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0
 0.0        0.0       0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0
 0.0        0.0       0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0
 0.0        0.0       0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0
 0.0        0.0       0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0

In [471]:
function jacobimult(mat,eivec,eival)

nrot = 0 
threshold = false
while threshold != true #Desde cero hasta que se alcance el threshold
    piv_r,piv_c = max_element(mat) #Busca el máximo elemento en el triangulo superior
    
    #EPS = .0000001
    
    if abs(mat[piv_r,piv_c]) < eps() || nrot > 50_000
            threshold = true #Sale del metodo
    

    else
        T = matrizT(mat,piv_r,piv_c) #Crea matriz T      
        eivec = eivec * T #Multiplica T con eigenvectores    
        mat = T' * mat * T #Gira matriz

        nrot = nrot + 1 #Contador de rotaciones
    end
end

eival = diag(mat) #Copia diagonal a eival
    
return mat,eivec,eival,nrot
end

jacobimult (generic function with 2 methods)

## Función crea matrices homogeneas

In [472]:
function creaMatHomogeneas(np)
    mat=zeros(np,np)
    #Crea matriz homogenea

    for i in 1:np
        k=ceil(np/2)
        for j in i:np        
            mat[i,j] = k
            mat[j,i] = k
            k=k-1
        end
    end
    return mat
end

creaMatHomogeneas (generic function with 1 method)

# Asignación de matrices

In [473]:
using LinearAlgebra
function asignaValorMatrices(np)
    #Matriz a rotar
    mat = creaMatHomogeneas(np)
    #Matriz eigenvectores
    eivec = Matrix(1.0I, np, np)
    #Vector eigenvalores
    eival = zeros(np)
    return mat, eivec, eival
end

asignaValorMatrices (generic function with 1 method)

# Driver

In [495]:
np = 10 #Orden de la matriz a evaluar

mat, eivec, eival = asignaValorMatrices(np)
mat,eivec,eival,nrot = jacobimult($mat,$eivec,$eival)

LoadError: syntax: "$" expression outside quote around In[495]:4

# Resultados

In [490]:
mat

10×10 Matrix{Float64}:
 20.4317       -1.64517e-16   5.31083e-25  …   1.03759e-20   3.9271e-31
 -4.94223e-16  20.4317       -6.35435e-25     -5.04879e-29  -4.93038e-32
  8.72298e-16  -6.56097e-17   2.42592         -2.57471e-23  -4.95101e-21
  1.85341e-16   9.50948e-16  -3.71609e-16     -1.03723e-29   1.82552e-23
 -1.33082e-16  -2.2445e-15   -1.45792e-16      2.60419e-18   3.48439e-18
 -5.09217e-16  -9.22909e-16  -8.22949e-17  …   8.05818e-18  -1.12606e-18
 -1.83027e-16   2.2369e-15    1.95342e-16      5.17877e-18  -6.12059e-17
  1.53103e-16  -9.65074e-17  -1.65554e-16      7.17871e-18  -1.19379e-17
  4.40736e-16  -1.51401e-15  -2.23495e-16      0.512543      3.59961e-17
 -1.05219e-15  -9.51911e-16   2.58634e-16      7.39484e-17   0.512543

In [491]:
eivec

10×10 Matrix{Float64}:
  0.441152   -0.0733806  -0.0931619  …   0.366514    0.0454835   0.444895
  0.442237    0.0665344  -0.408625      -0.422748   -0.180737   -0.409065
  0.400032    0.199937   -0.387206       0.130456    0.298299    0.333193
  0.318669    0.313768   -0.0465625      0.269388   -0.386662   -0.224706
  0.206113    0.396885    0.332468      -0.44714     0.437175    0.0942226
  0.0733806   0.441152    0.437402   …   0.256257   -0.444895    0.0454835
 -0.0665344   0.442237    0.181729       0.145892    0.409065   -0.180737
 -0.199937    0.400032   -0.223767      -0.427763   -0.333193    0.298299
 -0.313768    0.318669   -0.444783       0.356974    0.224706   -0.386662
 -0.396885    0.206113   -0.299107       0.0081155  -0.0942226   0.437175

In [492]:
eival

10-element Vector{Float64}:
 20.431729094530716
 20.43172909453074
  2.425919998159593
  2.4259199981595954
  1.0000000000000018
  0.9999999999999999
  0.6298080918412505
  0.6298080918412491
  0.5125428154684589
  0.5125428154684587

In [493]:
nrot

183